In [1]:
import csv
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

mc = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', save_best_only=True)  # tvalidation의 정확도 최대일때 저장
es = EarlyStopping(monitor='val_accuracy',mode='max',patience=5) 
# tvalidation의 정확도 최대일때. patience는 기준되는 값이 연속으로 몇 번 이상 향상되지 않을 때 종료시킬 것인지

In [2]:
df = pd.read_csv('C:/git_project/natural/data/mecab/X_train.csv')
df

,0,1,2,3,4,5,6,7,8,9,...,35,36,37,38,39,40,41,42,43,44
0,0,0,0,0,0,0,0,0,0,0,...,0,0,35,76,2528,45,245,22,39,717
1,0,0,0,0,0,0,0,0,0,0,...,698,4,122,1482,53,844,907,17,49,384
2,0,0,0,0,0,0,0,0,0,0,...,0,25,206,573,14,2,37,12,264,308
3,0,0,0,0,0,0,0,0,0,0,...,7131,161,4063,309,91,2,20,3,68,3391
4,0,0,0,0,0,0,0,0,0,0,...,61,10835,12697,7,421,9,1,4064,1613,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155934,12,4739,1120,19,15,1741,4,132,4,14,...,192,94,158,50,537,80,16,296,508,171
155935,0,0,0,0,0,0,0,0,0,0,...,4176,681,18,6,3,110,7,17220,49,3
155936,0,0,0,0,0,0,0,0,0,0,...,118,14,5,79,10,421,9,898,13,690
155937,0,0,0,0,0,0,0,0,0,0,...,9,1959,2,22,8,1407,421,9,105,171


In [3]:
X_train_data = pd.read_csv('C:/git_project/natural/data/mecab/X_train.csv')   # 데이터 불러오기
X_test_data = pd.read_csv('C:/git_project/natural/data/mecab/X_test.csv')
y_train_data = pd.read_csv('C:/git_project/natural/data/mecab/y_train.csv')   # 데이터 불러오기
y_test_data = pd.read_csv('C:/git_project/natural/data/mecab/y_test.csv')

In [4]:
X_train_data.dropna(how='any', inplace=True)  # null값 탈락
X_test_data.dropna(how='any', inplace=True)
y_train_data.dropna(how='any', inplace=True)
y_test_data.dropna(how='any', inplace=True)

In [5]:
train_X = np.array(X_train_data)       # array로 변경하여 tensor
test_X = np.array(X_test_data)
train_X, test_X

(array([[  0,   0,   0, ...,  22,  39, 717],
        [  0,   0,   0, ...,  17,  49, 384],
        [  0,   0,   0, ...,  12, 264, 308],
        ...,
        [  0,   0,   0, ..., 898,  13, 690],
        [  0,   0,   0, ...,   9, 105, 171],
        [  0,   0,   0, ..., 118, 608, 828]], dtype=int64),
 array([[   0,    0,    0, ...,    0,  825,  152],
        [   0,    0,    0, ..., 3358,   77,  930],
        [   0,    0,    0, ...,   32,   11,    2],
        ...,
        [   0,    0,    0, ...,    6,   15, 2260],
        [   0,    0,    0, ...,   66,  268,   46],
        [   0,    0,    0, ...,  177,   74, 1010]], dtype=int64))

In [6]:
train_y = np.array(y_train_data)
test_y = np.array(y_test_data)
train_y, test_y

(array([[0],
        [1],
        [0],
        ...,
        [1],
        [1],
        [1]], dtype=int64),
 array([[1],
        [0],
        [0],
        ...,
        [0],
        [0],
        [0]], dtype=int64))

# Model Building

In [13]:
vocab_size = 21854                                                   #전처리 이후 사이즈 확인

model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, 32, mask_zero=True))   # 임베딩 32
model.add(keras.layers.Dropout(0.5))                                 # dropout 0.2 - 0.5사이
model.add(keras.layers.Bidirectional(keras.layers.GRU(64)))          # 노드  64
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(16, activation='softsign'))             # softsign, tanh로 돌려봄
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, None, 32)          699328    
                                                                 
 dropout_6 (Dropout)         (None, None, 32)          0         
                                                                 
 bidirectional_2 (Bidirectio  (None, 128)              37632     
 nal)                                                            
                                                                 
 dropout_7 (Dropout)         (None, 128)               0         
                                                                 
 dense_4 (Dense)             (None, 16)                2064      
                                                                 
 dropout_8 (Dropout)         (None, 16)                0         
                                                      

In [14]:
model.compile(optimizer=keras.optimizers.Adam(0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Training

In [15]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train_data, y_train_data))
val_dataset = tf.data.Dataset.from_tensor_slices((X_test_data, y_test_data))

train_dataset = train_dataset.shuffle(buffer_size=1024, reshuffle_each_iteration=True).batch(128)  # batch_size 64,128,256 확인
val_dataset = val_dataset.batch(1000)

train_dataset = train_dataset.prefetch(4)

In [16]:
history = model.fit(train_dataset, epochs=10, validation_data=val_dataset, verbose=1,callbacks=[mc])
#callbacks로 체크포인트 확인, epochs 10,20회 반복으로 시도

Epoch 1/10
1219/1219 [==============================] - 201s 139ms/step - loss: 0.4214 - accuracy: 0.8090 - val_loss: 0.5279 - val_accuracy: 0.7288
Epoch 2/10
1219/1219 [==============================] - 140s 115ms/step - loss: 0.3552 - accuracy: 0.8529 - val_loss: 0.4526 - val_accuracy: 0.7763
Epoch 3/10
1219/1219 [==============================] - 117s 96ms/step - loss: 0.3306 - accuracy: 0.8666 - val_loss: 0.4032 - val_accuracy: 0.8083
Epoch 4/10
1219/1219 [==============================] - 127s 105ms/step - loss: 0.3105 - accuracy: 0.8761 - val_loss: 0.4097 - val_accuracy: 0.8116
Epoch 5/10
1219/1219 [==============================] - 118s 97ms/step - loss: 0.2951 - accuracy: 0.8847 - val_loss: 0.3978 - val_accuracy: 0.8193
Epoch 6/10
1219/1219 [==============================] - 130s 106ms/step - loss: 0.2832 - accuracy: 0.8885 - val_loss: 0.3725 - val_accuracy: 0.8310
Epoch 7/10
1219/1219 [==============================] - 111s 91ms/step - loss: 0.2712 - accuracy: 0.8947 - val_los